# Supervised Machine Learning Systems - (Classification)

In [1]:
# Helper functions to display a video or an image 
from IPython.display import HTML
def display_video(src):
    print('Source : '+src+ '?autoplay=1;modestbranding=1;rel=0')
    return HTML('<iframe width="800" height="400" src=' + src + '?autoplay=1;modestbranding=1;rel=0 frameborder="0" allowfullscreen></iframe>')

def display_image(src):
    print('Source : '+src)
    return HTML('<img width="600" height="300" src=' + src + '></img>')

## What is a Classification Problem ?

We briefly talked about Classification problem in our previous notebook.

1. **Independent Variables for classification** - These are also called features of our dataset. They are the variables which when varied can affect our target classes that we want to predict.
2. **Dependent Variable for classification** - When your target variable has certain class labels, its a classification problem. For instance classifying pictures of dogs and cats or a tumour to be cancerous or non cancerous etc. You are not predicting a continuous quantity here but different classes.

Lets take an example to understand it clearly :

<b> [Breast Cancer Diagnostic] </b>

There are two main classifications of tumors. One is known as benign and the other as malignant. A benign tumor is a tumor that does not invade its surrounding tissue or spread around the body. A malignant tumor is a tumor that may invade its surrounding tissue or spread around the body.

In [2]:
display_image('https://www.verywellhealth.com/thmb/xnYC1DVmfPtwjWCEdO0HjSZbcBo=/1787x0/filters:no_upscale():max_bytes(150000):strip_icc():format(webp)/514240-article-img-malignant-vs-benign-tumor2111891f-54cc-47aa-8967-4cd5411fdb2f-5a2848f122fa3a0037c544be.png')

Source : https://www.verywellhealth.com/thmb/xnYC1DVmfPtwjWCEdO0HjSZbcBo=/1787x0/filters:no_upscale():max_bytes(150000):strip_icc():format(webp)/514240-article-img-malignant-vs-benign-tumor2111891f-54cc-47aa-8967-4cd5411fdb2f-5a2848f122fa3a0037c544be.png


Our target it to train a Logistic Regression model that can predict whether the cancer is benign (B) or malignant (M).

Attribute Information:
<br>1) ID number 
<br>2) Diagnosis (M = malignant, B = benign) 
<br>3-32) Ten real-valued features are computed for each cell nucleus: 
<br>a) radius (mean of distances from center to points on the perimeter) 
<br>b) texture (standard deviation of gray-scale values) 
<br>c) perimeter 
<br>d) area 
<br>e) smoothness (local variation in radius lengths) 
<br>f) compactness (perimeter^2 / area - 1.0) 
<br>g) concavity (severity of concave portions of the contour) 
<br>h) concave points (number of concave portions of the contour) 
<br>i) symmetry 
<br>j) fractal dimension ("coastline approximation" - 1)

**`'Diagnosis'`** column is the **Dependent Variable or target column** because we want our algorithm to predict this class.

**`'1,3-32'`** are your **Features or Independent Variables** which will help you predict the Benign/Malignant class. Vary any one of them and it is going to affect your Diagnostic.

## Basic Intuition 

Now we will discuss about the Logistic Regression algorithm. Don't be confused by the name "Logistic Regression"; it is named that way for historical reasons and is actually an approach to classification problems, not regression problems.

Instead of our output vector y being a continuous range of values, it will only be 'M' or 'B'.

In [3]:
# Importing required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Read the CSV file  ../Data/Breast_Cancer_Diagnostic.csv

In [4]:
# Loding the dataset into pandas dataframe.
df = pd.read_csv('https://github.com/diksha-cl/Data-files/blob/master/Breast_Cancer_Diagnostic.csv')

In [5]:
# print all the available features.
df.columns

Index(['id', 'diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst', 'Unnamed: 32'],
      dtype='object')

In [6]:
# We will use only 10 of the 30 odd features. To get a feel for the features, 
# take a sample from the dataset.
sample = df.sample(n=5, random_state=10)

In [7]:
# print the first five features that we will use and the target variable from the sample.
sample[["radius_mean", "texture_mean", "perimeter_mean", "area_mean", "smoothness_mean", 
        "diagnosis"]]

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,diagnosis
172,15.460,11.89,102.50,736.9,0.12570,M
553,9.333,21.94,59.01,264.0,0.09240,B
374,13.690,16.07,87.84,579.1,0.08302,B
370,16.350,23.29,109.00,840.4,0.09742,M
419,11.160,21.41,70.95,380.3,0.10180,B


In [8]:
# print the next five features from the same sample.
sample[['compactness_mean', 'concavity_mean',
    'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean','diagnosis']]

,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,diagnosis
172,0.15550,0.203200,0.10970,0.1966,0.07069,M
553,0.05605,0.039960,0.01282,0.1692,0.06576,B
374,0.06374,0.025560,0.02031,0.1872,0.05669,B
370,0.14970,0.181100,0.08773,0.2175,0.06218,M
419,0.05978,0.008955,0.01076,0.1615,0.06144,B


*We will only consider ten real-valued features in this project for diagnostic!<br>
Let's separate the required features along with diagnosis column.*

In [9]:
# Retain the 10 features and the target variable.
df = df[['radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean','diagnosis']]

In [10]:
# Check for nulls.
df.columns[df.isnull().any()]

Index([], dtype='object')

In [11]:
# Count the number of malignants and benigns in the dataset.
df['diagnosis'].value_counts()

B    357
M    212
Name: diagnosis, dtype: int64

## Create the Dataframe of features (X) and the target (Y) variables

In [12]:
# Load the features to a variable X
# X is created by simply dropping the diagnosis column and retaining all others
X = df.drop('diagnosis', axis = 1)

# Load the target variable to y
y = df['diagnosis']

## Split Test Train

**> Train-Test split -** We split our data into two parts, namely, the train set and the test set (ideally its a 70-30 train-test split which is upto you). We then try to build our function f(x) (aka model) using the train set and see how well it does on the test set.   

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1)

## Create an Instance of the classifier and train it.

In [14]:
# Let's create an instance for the LogisticRegression model and then train it with the training set.
from sklearn.linear_model import LogisticRegression
Classifier = LogisticRegression()
Classifier.fit(X_train,y_train)

C:\Users\subhrajit\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

## Get the Predictions

In [15]:
# Getting predictions from the model 
y_test_hat = Classifier.predict(X_test)

# Compare the predicted values with the actuals.
Results = pd.DataFrame({'Actual': y_test})
column = pd.DataFrame({'Predictions': y_test_hat})
Results = Results.join(column.set_index(Results.index))
Results.head(5)

,Actual,Predictions
421,B,B
47,M,B
292,B,B
186,M,M
414,M,M


### 1. The accuracy scores

It is simply calculated as number of classes predicted right divided by total number of samples.

In [16]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_test_hat))

0.8888888888888888


In [17]:
# Get the predictions from the model for the training set. 
y_train_hat = Classifier.predict(X_train)
print(accuracy_score(y_train, y_train_hat))

0.9120603015075377
